In [160]:
import os
os.getcwd()

'D:\\Python\\finominaScraping'

In [161]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np

In [162]:
def transformSET(path:str,QUARTER_REF:int):
    SET_RAWST = pd.read_csv(path)

    dateSplit = pd.DataFrame()
    dateSplit[['day','quarter','year']]=SET_RAWST['Timestamp'].str.split("-",n=0,expand=True)
    dateSplit['year'] = pd.to_datetime(dateSplit['year'].str.strip().astype(str), format='%y').dt.strftime('%Y')
    dateSplit['year'] = dateSplit['year'].astype(int)
    dateSplit.loc[  (dateSplit["quarter"]  == "01" ) , 'year'] = dateSplit['year']-1
    dateSplit['quarter']=dateSplit['quarter'].replace("01","4")
    dateSplit['quarter']=dateSplit['quarter'].replace("04","1")
    dateSplit['quarter']=dateSplit['quarter'].replace("07","2")
    dateSplit['quarter']=dateSplit['quarter'].replace("10","3")
    SET_RAWST = pd.concat([dateSplit,SET_RAWST[['SET.Close','Vol']]   ], axis=1) 

    #take pct_change
    SET_RAWST = SET_RAWST.sort_values(['year','quarter'],ascending=True)
    SET_RAWST['Vol'] = SET_RAWST['Vol'].apply(lambda x : float( x.replace(",","") )).pct_change()
    SET_RAWST['SET'] = SET_RAWST['SET.Close'].apply(lambda x : float( x.replace(",","") )).pct_change()

    SET_RAWST = SET_RAWST.sort_values(['year','quarter'],ascending=False)
    SET_RAWST['Date'] = np.add(   
            np.add(SET_RAWST['quarter'].astype(str),["Q"]),
            SET_RAWST['year'].astype(str) )

    
    SET_RAWST = SET_RAWST[['Date','SET','Vol']].set_index("Date")    
    SET_STG     = pd.DataFrame()
    for c in SET_RAWST.columns:
        for ref in range(0,QUARTER_REF+1):
            SET_STG[f'{c}[{0-ref}]'] = SET_RAWST[c].shift(-ref)
        
    return SET_STG

In [183]:
def transformSTG_X(path:str,QUARTER_REF:str=5,path_set:str="./data/SET.BK.csv"):
    
    DF_RWST = pd.read_csv(path)
    DF_RWST = DF_RWST.rename(columns={
        'Unnamed: 0'            : "Date", 
        'รายได้รวม'              : "TotalRevenue", #*
        'การเติบโตต่อไตรมาส (%)'  : "growth/quarter",
        'การเติบโตต่อปี (%)'       : "growth/year",
        'กำไรสุทธิ'               : "NetProfit",   #*
        'กำไรต่อหุ้น (EPS)'        : "EPS",
        'ROA (%)'               : "ROA",
        'ROE (%)'               : "ROE",
        'อัตรากำไรขั้นต้น (%)'      : "GrossProfitMargin",
        'อัตราส่วนการขายและบริหารต่อรายได้ (%)'  : "OperatingMargin",
        'อัตรากำไรสุทธิ (%)'        : "NetProfitMargin",
        'หนี้สิน/ทุน (เท่า)'         : "debt/capital",
        'วงจรเงินสด (วัน)'          : "CashConversionCycle",
        'มูลค่าหลักทรัพย์ตามราคาตลาด (ล้านบาท)'  : "MKT.values",
        'P/E (เท่า)'                : "P/E",
        'P/BV (เท่า)'               : "P/BV",
        'มูลค่าหุ้นทางบัญชีต่อหุ้น (บาท)' : "BookValue/Share",
        'อัตราส่วนเงินปันผลตอบแทน'    : "DividendYield", #drop
        'EV / EBITDA'              : "EV/EBITDA",
        'ราคาล่าสุด (บาท)'           : "Price",     #*
    })
    
    #adj Price with Dv
    DF_RWST['Price'] = DF_RWST['DividendYield'] + DF_RWST['Price']
    #drop Dv
    DF_RWST = DF_RWST.drop('DividendYield',axis=1)
    
    DF_RWST['year'] = DF_RWST['Date'].str.slice(start=2)
    DF_RWST['quarter'] = DF_RWST['Date'].str.slice(stop=1)
    
    #take pct_change
    DF_RWST = DF_RWST.sort_values(['year','quarter'],ascending=True)
    
    DF_RWST['MKT.values'] = DF_RWST['MKT.values'].apply(lambda x : float( x.replace(",","") ))
    DF_RWST["MKT.values"] = DF_RWST["MKT.values"].pct_change()
    
    DF_RWST["TotalRevenue"] = DF_RWST["TotalRevenue"].pct_change()
    DF_RWST["NetProfit"] = DF_RWST["NetProfit"].pct_change()
    DF_RWST["Price"] = DF_RWST["Price"].pct_change()
    
    #transform
    DF_RWST = DF_RWST.sort_values(['year','quarter'],ascending=False)

    DF_Cross = DF_RWST[ ["Symbol","Market","IndusGroup","BusinessGroup"] ]
    
    ##### transformDate #####   
    DF_Date     = pd.DataFrame()
    DF_Date['year']   = DF_RWST['year']
    DF_Date['quarter']= DF_RWST['quarter']
    
    #--tran พศ to คศ  
    year = DF_RWST['year'].to_numpy()
    year = year.astype(int) - 543
    DF_Date['Date'] = np.add(   
        np.add(DF_RWST['quarter'].astype(str),["Q"]),
        year.astype(str) )

    DF_Ts    = DF_RWST.drop( columns=list(DF_Cross.columns)+list(DF_Date.columns) )
    DF_Ts    = DF_Ts.reset_index(drop=True)
    
    ##### transformDF #####
    STG_X = pd.DataFrame()
    for ref in range(1,QUARTER_REF+1):
        STG_X = pd.concat([STG_X ,DF_Ts.shift(-ref).add_suffix(f'[{-ref}]')], axis=1)
        
    STG_X = pd.concat([STG_X,DF_Ts.add_suffix(f'[0]')   ], axis=1) 
    #----
    STG_X = STG_X.reset_index(drop=True)
    
    # ADD "year","quarter"
    # STG_symbol = pd.concat([STG_symbol,DF_Date[ ["year","quarter"] ]   ], axis=1) 

    # ADD CrossSection data
    STG_X = pd.concat([STG_X,DF_Cross   ], axis=1) 
    
    DF_Date = DF_Date.reset_index()
    STG_X['Date'] = DF_Date['Date']
    STG_X = STG_X.set_index('Date')
    
    
    #join
    SET_STG = transformSET(path_set,QUARTER_REF)
    STG_X.merge(  SET_STG,on="Date" )
    
    return STG_X ,DF_Date.set_index('Date')
    

In [184]:
def transformSTG_Y(STG_X,Date):
    Y={
        'Price[1]':[np.nan]*1, #1q
        'Price[2]':[np.nan]*2, #2q
        'Price[3]':[np.nan]*3, #3q
        'Price[4]':[np.nan]*4  #4q
    }

    price = STG_X['Price[0]'].values

    for i in range(len(STG_X)):
        for j in range( 1,len(Y)+1 ):
            if i >= j:
                Y[f'Price[{j}]'] += [price[i-j]]    
                
    Y = pd.DataFrame(Y)
    Y['Price[0]'] = STG_X['Price[0]'].values
    Y = Y[Y.columns.sort_values()]
    Y['Date'] = Date.index
    return Y.set_index("Date")

In [185]:
def ratingScore(STG_Y):
    STG_Y['Score'] = 0
    
    STG_Y.loc[ (STG_Y["Price[1]"] > STG_Y["Price[0]"]), 'Score'] = STG_Y['Score']+1
    STG_Y.loc[ (STG_Y["Price[1]"] > STG_Y["Price[0]"]) & (STG_Y["Price[2]"] > STG_Y["Price[1]"]), 'Score'] = STG_Y['Score']+1
    STG_Y.loc[ (STG_Y["Price[1]"] > STG_Y["Price[0]"]) 
              & (STG_Y["Price[2]"] > STG_Y["Price[0]"])
              & (STG_Y["Price[2]"] > STG_Y["Price[1]"]), 'Score'] = STG_Y['Score']+1
    
    #add Score
    STG_Y.loc[  (STG_Y["Price[1]"]  >=0.10) , 'Score'] = STG_Y['Score']+1
    STG_Y.loc[  (STG_Y["Price[1]"]  >=0.10) , 'Score'] = STG_Y['Score']+1
    STG_Y.loc[  (STG_Y["Price[2]"]  >=0.10) , 'Score'] = STG_Y['Score']-1
    STG_Y.loc[  (STG_Y["Price[2]"]  >=0.10) , 'Score'] = STG_Y['Score']-1
    
    #set Score nan
    STG_Y.loc[  STG_Y["Price[1]"].isnull()  , 'Score'] = np.nan
    STG_Y.loc[  STG_Y["Price[2]"].isnull()  , 'Score'] = np.nan
    #STG_Y.loc[  STG_Y["Price[0]"].isnull()  , 'Score'] = np.nan
    return STG_Y[['Score']]

In [186]:
# RWST => transformSTG_Y => STG_X,DATE
STG_X,Date = transformSTG_X('./data/RWST/MFC.csv',)

In [187]:
# STG_X => transformSTG_Y => STG_Y
STG_Y = transformSTG_Y(STG_X,Date)

In [188]:
# STG_Y => Score => score_Y
score_Y = ratingScore(STG_Y)

In [189]:
score_Y.head()

,Score
Date,
3Q2022,NaN
2Q2022,NaN
1Q2022,0.0
4Q2021,3.0
3Q2021,3.0


In [ ]:
#==> drop Symbol
#==> one hot : Market
#==> one hot : IndusGroup
#==> one hot : BusinessGroup
